In [0]:
from pyspark.sql import functions as F
from pyspark.sql.window import Window

df_prata = df_bronze.withColumnRenamed("lunch", "Tipo_Almoco")

w = Window.orderBy(F.monotonically_increasing_id())
df_prata = df_prata.withColumn("ID_Estudante", F.row_number().over(w))

cols = ["ID_Estudante"] + [c for c in df_prata.columns if c != "ID_Estudante"]
df_prata = df_prata.select(*cols)

display(df_prata.limit(5))
df_prata.printSchema()

colunas_categoricas = ["Genero", "Etnia", "Educacao_dos_pais", "Curso_Preparatorio", "Tipo_Almoco"]

for c in colunas_categoricas:
    df_prata = df_prata.withColumn(c, F.trim(F.lower(F.col(c))))
for c in colunas_categoricas:
    print(f"Valores distintos de {c}:")
    display(df_prata.groupBy(c).count())

# Garantir que notas são inteiras
df_prata = (
    df_prata
        .withColumn("Nota_Mat", F.col("Nota_Mat").cast("int"))
        .withColumn("Nota_Leitura", F.col("Nota_Leitura").cast("int"))
        .withColumn("Nota_Escrita", F.col("Nota_Escrita").cast("int"))
)

# Criar coluna de média das 3 notas
df_prata = df_prata.withColumn(
    "Nota_Media",
    ((F.col("Nota_Mat") + F.col("Nota_Leitura") + F.col("Nota_Escrita")) / 3.0)
)

display(df_prata.limit(5))
df_prata.printSchema()

(
    df_prata
      .write
      .mode("overwrite")
      .format("delta")
      .saveAsTable("Camada_Prata_Estudantes")
)
df_prata_tabela = spark.table("Camada_Prata_Estudantes")

display(df_prata_tabela.limit(10))
df_prata_tabela.printSchema()


/databricks/python/lib/python3.12/site-packages/pyspark/sql/connect/expressions.py:1134: UserWarning: WARN WindowExpression: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
  warnings.warn(


ID_Estudante,Genero,Etnia,Educacao_dos_pais,Tipo_Almoco,Curso_Preparatorio,Nota_Mat,Nota_Escrita,Nota_Leitura
1,female,group B,bachelor's degree,standard,none,72,74,72
2,female,group C,some college,standard,completed,69,88,90
3,female,group B,master's degree,standard,none,90,93,95
4,male,group A,associate's degree,free/reduced,none,47,44,57
5,male,group C,some college,standard,none,76,75,78


root
 |-- ID_Estudante: integer (nullable = false)
 |-- Genero: string (nullable = true)
 |-- Etnia: string (nullable = true)
 |-- Educacao_dos_pais: string (nullable = true)
 |-- Tipo_Almoco: string (nullable = true)
 |-- Curso_Preparatorio: string (nullable = true)
 |-- Nota_Mat: integer (nullable = true)
 |-- Nota_Escrita: integer (nullable = true)
 |-- Nota_Leitura: integer (nullable = true)

Valores distintos de Genero:


Genero,count
female,518
male,482


Valores distintos de Etnia:


...

Tipo_Almoco,count
standard,645
free/reduced,355


ID_Estudante,Genero,Etnia,Educacao_dos_pais,Tipo_Almoco,Curso_Preparatorio,Nota_Mat,Nota_Escrita,Nota_Leitura,Nota_Media
1,female,group b,bachelor's degree,standard,none,72,74,72,72.66666666666667
2,female,group c,some college,standard,completed,69,88,90,82.33333333333333
3,female,group b,master's degree,standard,none,90,93,95,92.66666666666667
4,male,group a,associate's degree,free/reduced,none,47,44,57,49.333333333333336
5,male,group c,some college,standard,none,76,75,78,76.33333333333333


root
 |-- ID_Estudante: integer (nullable = false)
 |-- Genero: string (nullable = true)
 |-- Etnia: string (nullable = true)
 |-- Educacao_dos_pais: string (nullable = true)
 |-- Tipo_Almoco: string (nullable = true)
 |-- Curso_Preparatorio: string (nullable = true)
 |-- Nota_Mat: integer (nullable = true)
 |-- Nota_Escrita: integer (nullable = true)
 |-- Nota_Leitura: integer (nullable = true)
 |-- Nota_Media: double (nullable = true)



ID_Estudante,Genero,Etnia,Educacao_dos_pais,Tipo_Almoco,Curso_Preparatorio,Nota_Mat,Nota_Escrita,Nota_Leitura,Nota_Media
1,female,group b,bachelor's degree,standard,none,72,74,72,72.66666666666667
2,female,group c,some college,standard,completed,69,88,90,82.33333333333333
3,female,group b,master's degree,standard,none,90,93,95,92.66666666666667
4,male,group a,associate's degree,free/reduced,none,47,44,57,49.333333333333336
5,male,group c,some college,standard,none,76,75,78,76.33333333333333
6,female,group b,associate's degree,standard,none,71,78,83,77.33333333333333
7,female,group b,some college,standard,completed,88,92,95,91.66666666666667
8,male,group b,some college,free/reduced,none,40,39,43,40.666666666666664
9,male,group d,high school,free/reduced,completed,64,67,64,65.0
10,female,group b,high school,free/reduced,none,38,50,60,49.333333333333336


root
 |-- ID_Estudante: integer (nullable = true)
 |-- Genero: string (nullable = true)
 |-- Etnia: string (nullable = true)
 |-- Educacao_dos_pais: string (nullable = true)
 |-- Tipo_Almoco: string (nullable = true)
 |-- Curso_Preparatorio: string (nullable = true)
 |-- Nota_Mat: integer (nullable = true)
 |-- Nota_Escrita: integer (nullable = true)
 |-- Nota_Leitura: integer (nullable = true)
 |-- Nota_Media: double (nullable = true)

